# Implementation of the EBfilter by genomon

* EBrun (originally EBFilter) is an argparse wrapper passing command line arguments to run.py (is not needed for internal use)
* passed arguments:
    * targetMutationFile: the .vcf or .anno containing the mutations
    * targetBamPath: path to the tumor bam file (+.bai)
    * controlBamPathList: text list of path to PoN bam files (+ .bai)
    * outputPath
    * -q option for quality threshold – not needed --> default _q config
    * -Q option for base quality threshold - not needed --> default _Q from config
    * --region option for restriction of regions on mpileup -l

## Initiation

### imports

In [1]:
import os
from code import run

### snakemake config

In [2]:
config = {'EB':{'run': True}}
params = {}
params['map_quality'] = 20
params['base_quality'] = 15
params['filter_flags'] = 'UNMAP,SECONDARY,QCFAIL,DUP'
params['loption'] = True
config['EB']['threads'] = 1
config['EB']['params'] = params
config['annovar'] = {'sep': '\t'}

### load the config and GLOBAL STATE into variable _

In [3]:
debug_mode = True
args = {}
params = config['EB']['params']
threads = config['EB']['threads']
sep = config['annovar']['sep']
_q = str(params['map_quality'])  # mapping quality=20
_Q = params['base_quality']      # base quality=15
filter_quals = ''
for qual in range( 33, 33 + _Q ): 
    filter_quals += chr( qual )  # qual asciis for filtering out
_ff = params['filter_flags']     # 'UNMAP,SECONDARY,QCFAIL,DUP'
is_loption = params['loption']   # True
log_path = 'output/logs'         # log_file: 'output/logs/'



### function args
will be passed during the function call

In [4]:
HOME = '/Users/mahtin'
args['mut_file'] = f'{HOME}/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/anno/test_rel.csv'
args['tumor_bam'] = f'{HOME}/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/bam/test_Rel1.bam'
args['pon_list'] = f'{HOME}/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/aml_pon.list'
args['output_path'] = 'output/test_rel_eb.csv'
args['region'] = ''
log_file = os.path.join(log_path, f"{os.path.splitext(os.path.basename(args['output_path']))[0]}.log")
filter_quals = '!"#$%&\'()*+,-./'
# dump into _
_ = {'q':_q, 'Q':_Q, 'filter_quals': filter_quals, 'ff':_ff, 'log':log_file, 'threads':threads, 'sep': sep, 'debug_mode':debug_mode}

# Using dataframes

In [5]:
##### RUNNIT
_['threads'] = 1
_['debug_mode'] = True
run.main(args,_)
!ls output

index:  5 read:  tt-3nnnTTTtTTTT
index:  5 read:  ttTTTtTTTT
index:  8 read:  TtgtgT-3NNNTTTT
index:  8 read:  TtgtgTTTTT


NameError: name 'EB_score' is not defined

## transforming for EB_score

## getting the count matrix from a df row

In [34]:
import pandas as pd
from functools import partial

mut_df = pd.read_csv('output/test_rel_clean.csv', sep='\t')
# for snps
row = mut_df.iloc[15]

def get_count(var, read):
    values = [read.count(r'[ACTG]')]
    values.append(read.count(f'{var.upper()}'))
    values.append(read.count(r'[acgt]'))
    values.append(read.count(f'{var.lower()}'))
    count_matrix = pd.Series(values, index=['depth_p', 'mismatch_p', 'depth_n', 'mismatch_n'])
    return count_matrix
var = row['Alt']
values = [row.iloc[6::3].str.count(r'[ACTG]')]
values

[read0     44
 read1     10
 read2      8
 read3     10
 read4      8
 read5     13
 read6     14
 read7      9
 read8     11
 read9     17
 read10    17
 Name: 15, dtype: int64]

### adjusting for bad quality calls

In [ ]:
import re
filters = '!"#$%?&\'()*+,-./'
filter_string = r"([" + filters + "])"
filter_re = re.compile(filter_string)
read = 'AAAA*BBB'
Q = 'AAAA*BBB'
while filter_re.search(Q):
    m = filter_re.search(Q)
    print(Q,read, m.end(), m.group(0), m.group(1))
    Q = filter_re.sub('', Q, count=1)
    pos = max(m.start(),m.end()-1)
    read = read[:pos] + read[pos+1:]
print(read)


In [ ]:
import pandas as pd
import re
from functools import partial

filters = '!"#$%?&\'()*+,-./'
filter_string = r"([" + filters + "])"
filter_re = re.compile(filter_string)

def remove_badQ(i,row):
    Q = row[f"Q{i}"]
    read = row[f"read{i}"]
    while filter_re.search(Q):
        print(read, Q)
        m = filter_re.search(Q)
        Q = filter_re.sub('', Q, count=1)
        pos = max(m.start(),m.end()-1)
        read = read[:pos] + read[pos+1:]
    print(read,Q)
    return row   
mut_df = pd.read_csv('output/test_rel_clean.csv', sep='\t')
pon_count = 10

is_snp = (mut_df['Ref'] != '-') & (mut_df['Alt'] != '-')
# example for row 7
has_badQ = mut_df['Q0'].str.contains(filter_re)
bad_df = mut_df[is_snp & has_badQ]
mut_df[is_snp & has_badQ] = bad_df.apply(partial(remove_badQ, 0), axis=1)


In [ ]:
def remove_badQ(df, filters):
    filter_string = r"[" + filters + "]"
    filter_re = re.compile(filter_string)
    is_snp = (df['Ref'] != '-') & (df['Alt'] != '-')
    for i in range(pon_count + 1):
        Q = f"Q{i}"
        print(Q)
        bad_rows = df[snp][df.str.contains(filter_re)]
        print(bad_rows)


In [ ]:
import re

filter_string = r"(^[" + filters + "].)|(.[" + filters + "])"
filter_re = re.compile(filter_string)
test = "s*d"
m = filter_re.search(test)
m.group()

In [ ]:
import pandas as pd
mut_df = pd.read_csv('output/test_rel_clean.csv', sep='\t')
mut_df[:13]

### letter count

In [ ]:
import pandas as pd
mut_df = pd.read_csv('output/test_rel_clean.csv', sep='\t')
mut_df
# check length:
for i in range(11):
    read = f"read{i}"
    Q = f"Q{i}"
    test = mut_df[mut_df[read].str.len() != mut_df[Q].str.len()]
    print(len(test.index))
i = 7
read = f"read{i}"
Q = f"Q{i}"
test = mut_df[mut_df[read].str.len() != mut_df[Q].str.len()]
test

### cleaning the data

In [ ]:
import pandas as pd
import re
from functools import partial

mut_df = pd.read_csv('output/test_rel_eb.csv', sep='\t')

# remove start_end signs in reads
sign_re = re.compile(r'\^.|\$')
indel_simple = re.compile(r'[\+\-]([0-9]+)')

mut_df['read'] = mut_df['read'].str.replace(sign_re, '')
for i in range(10):
    read = f"read{i}"
    mut_df[read] = mut_df[read].str.replace(sign_re, '')
is_indel = (mut_df['Ref'] == '-') | (mut_df['Alt'] == '-')
def clean_indels(i, row):
    indel_length = indel_simple.search(row['read']).group(1)
    re_string = r"([ACGTNacgtn])([\+\-])([0-9]+)([ACGTNacgtn]{" + str(indel_length) + "})"
    indel_re = re.compile(re_string)
    row['read'] = indel_re.sub(r'\1', row['read'])
    for i in range(i):
        read = f"read{i}"
        row[read] = indel_re.sub(r'\1', row[read])
    return row
clean_indels_i = partial(clean_indels, 10)
    
mut_df[is_indel] = mut_df[is_indel].apply(clean_indels_i, axis=1)
mut_df[is_indel]

# mut_df['read'].replace(sign_re, '')




## importing pileup directly into dataframe

In [ ]:
import pandas as pd
from io import StringIO
import subprocess
with open('output/testpile', 'w') as file_out:
    bam = 'testdata/tumor.bam'
    cmd = ["samtools", "mpileup", "-B", "-d", "10000000", bam]
    pileup = subprocess.Popen(cmd, stdout=subprocess.PIPE)
b = StringIO(pileup.communicate()[0].decode('utf-8'))
pile_df = pd.read_csv(b, sep='\t', header=None, names=['Chr', 'Start', 'Alt', 'depth', 'read', 'quality'])
pile_df[:14]

### using apply on the whole dataframe

In [ ]:
def useit(row):
    return f"{row['Alt']} and {row['read']}"

    
pile_df.apply(useit, axis=1)[:13]

In [ ]:
import pandas as pd
test = pd.DataFrame(['abc', 'aaaccd'],['aaccaa', 'aaatccca'])

In [ ]:
test[0,0].value_counts()

## joining pileup and mut_df into one

In [ ]:
import pandas as pd
mut_df = pd.read_csv('/Users/mahtin/Dropbox/Icke/Work/somVar/tools/EBFilter/mytestdata/anno/test_rel.csv', sep='\t').sort_values(['Chr', 'Start'])
mut_df['Start'] -= (mut_df['Alt'] == '-')
pileup_df = pd.read_csv('output/test_rel_eb.target.pileup', header=None, sep='\t', names=['Chr', 'Start', 'Ref', 'depth', 'read', 'Q'], dtype = {'Chr':int, 'Start':int, 'Ref':str, 'depth':int, 'read':str, 'Q':str}).drop(columns='Ref')
pileup_df
mut_df = pd.merge(left=mut_df, right=pileup_df, on=['Chr', 'Start'], how='outer', left_index=True)
# control
names = ['Chr', 'Start','Ref']
for i in range(10):
    names += [f"depth{i}", f"read{i}", f"Q{i}"]
control_df = pd.read_csv('output/test_rel_eb.control.pileup', header=None, sep='\t', names = names).drop(columns='Ref')
mut_df = pd.merge(left=mut_df, right=control_df, on=['Chr', 'Start'], how='outer', left_index=True)
mut_df
# mut_df['Start'] += (mut_df['Alt'] == '-')

In [ ]:
names = ['Chr', 'Start','Ref']
for i in range(10):
    names += [f"depth{i}", f"read{i}", f"Q{i}"]
names

In [ ]:
mut_df

## Single threading

In [ ]:
##### RUNNIT
_['threads'] = 1
_['debug_mode'] = False
main(args,_)
!ls output

## Multithreading

In [ ]:
_['threads'] = 3
_['debug_mode'] = False
run.main(args,_)
!ls output/

### trying with multiprocessing Pool

In [ ]:
from functools import partial
from multiprocessing import Pool
def show(a,b,c,d):
    return a + b + c +d
a = 'a'
b = 'b'
c = 'c'
itera = ['d', 'e', 'f']
show_part = partial(show, a, b, c)
show_part('t')
pool = Pool(3)
result = pool.map(show_part, itera)
pool.close()
result
